<tagname>
    <H1>Assignment 3</H1>
    <p>For my standard Naive Bayes algorithm, I identify 1000 frequently occurring words and generate a word dictionary which contain word name and ID number (from 1 to 1000). Then create a word metrics which have 1000 columns and each column count the number of words occurs in the corresponding word dictionary.  In order to avoid the zero-value problem when calculating the proportion of a certain word in a certain category. I added Laplace smoothing. The numerator will plus the coefficient 1 and the denominator plus the size of the whole dictionary which is 1000.</p>
    <p>I created four lists to store the number of unique word occours in Class A, B, E and V respectively. According to the assumption of Naive Bayes that each attribute is independent of each other, the text of abstracts can be viewed as a sequence composed of many words, and the words in the abstract can be used to build a polynomial model. Finally, the multiplication of the probability of each word is rewritten into the mode of log (probability) summation to optimize our Naive Bayes algorithm.Then return the highest-ranking class through sorting. This is the process how I create the trainning model. In my first version I did not include pre-processing of the texts. After apply a 10 folds cross-validation results in a score of 90.5%. </p>
    <p>“Stop words” in the text such as' am','is','are','was','were','she','he'. We need to delete those words that are not helpful for the further analysis of the article in the pre-processing stage. Moreover, the repeated occurrence of stop words is highly likely to cause model misclassification. After deleting the “stop words”, the accuracy was slightly improved, reaching 90.7%.</p>
    <p>Different attribute-value affect the accuracy a lot. My next extension is to obtain an attribute-value representation that identifies 12000 unique words in the training set. Adjust the number of selected most frequency occurring words through the results of 10 folds cross-validation. I found that the accuracy increased with the increase of the number of unique words. When about 12,000 most frequently occurring words were selected as unique words vector, the cross-validation result seems to be the maximum. After this threshold is exceeded, the accuracy begins to decrease. Compared with only choose 1000 most frequently occurring words, the accuracy of this predicting model raised significantly which score is 95.525%</p>
    <p>In my final extension, I used a method called TF-IDF to improve the accuracy of the model. This method optimizes the algorithm by giving weights to words. If a word appears in many paragraphs which IDF value should be lower. On the other hand, if a word appears relatively few occurrences, its IDF value should be high. The formula is log (Y / W + 1) where Y is the total number of documents in the whole CSV file, and W is the number of documents containing the term. In the end, the prediction model obtained by multiplying TF and IDF which makes the model more accurate. The accuracy of the final extended version reaches 96.225%</p>
</tagname>

In [3]:
import numpy as np

In [79]:
data = np.genfromtxt(f'trg.csv', delimiter=',', encoding='utf8', dtype=None)[1:]
train_filename = "trg"
test_filename = "tst"
output_filename = "eliu092"

In [80]:
def write_output(test_filename, output_filename, trained_model):
    test_data = np.genfromtxt(f'{test_filename}.csv', delimiter=',', encoding='utf8', dtype=None)[1:]
    output = open(f'{output_filename}.csv', encoding='utf8', mode='w')
    output.write('id,class\n')
    for i in range(len(test_data)):
        abstract = test_data[i][1].replace('"', ' ').split()
        prediction = predict(abstract, trained_model)
        output.write(f'{i + 1},{prediction}\n')
    output.close()

def predict(abstract,trained_model):
        top1000_dictionary_id, label_prob, conditional_probabilities_A,conditional_probabilities_B,conditional_probabilities_E,conditional_probabilities_V = trained_model
        word_vector = np.zeros(1000, dtype=int)
        for word in abstract:
            if word in top1000_dictionary_id:
                word_vector[top1000_dictionary_id[word]] += 1
                
        
        p_label_A = np.log(label_prob['A'])
        p_A = sum(word_vector * np.log(conditional_probabilities_A)) + p_label_A
    
        p_label_B = np.log(label_prob['B'])
        p_B = sum(word_vector * np.log(conditional_probabilities_B)) + p_label_B
    
        p_label_E = np.log(label_prob['E'])
        p_E = sum(word_vector * np.log(conditional_probabilities_E)) + p_label_E
    
        p_label_V = np.log(label_prob['V'])
        p_V = sum(word_vector * np.log(conditional_probabilities_V)) + p_label_V
        prediction = {"A": p_A, "B": p_B, "E": p_E, "V": p_V}
        prediction_array = prediction.items()
        p = sorted(prediction_array, key=lambda key_value: key_value[1], reverse=True)
        p = p[0]
        return p[0]

def Convert(tup, dictionary):
    i = 0
    for a, b in tup:
        dictionary.setdefault(a,i)
        i += 1
    return dictionary

def NaiveBayes(data):
    count_apperance = {}
    top1000 = []
    top1000_dictionary = {}
    for i in range(len(data)):
        abstract =  data[i][2]
        abstract = abstract.replace('"', ' ').split()
        for word in abstract:
            if word in count_apperance:
                count_apperance[word] += 1
            else:
                count_apperance[word] = 1
    count_apperance_array = count_apperance.items()    
    descending_sorting = sorted(count_apperance_array, key = lambda x:x[1], reverse = True)
    for i in range(0,1000):
        top1000.append(descending_sorting[i])
    top1000_dictionary_id =  Convert(top1000, top1000_dictionary)
    new_data = np.zeros((len(data), 1000))
    for i in range(len(data)):
        abstract = data[i][2].split()
        word_vector = np.zeros(1000, dtype=int)
        for word in abstract:
            if word in top1000_dictionary_id:
                word_vector[top1000_dictionary_id[word]] += 1
        new_data[i] = word_vector
# class probability calculation
    label_count = {}
    for i in range(len(data)):
        label = data[i][1]
        if label in label_count:
            label_count[label] += 1
        else:
            label_count[label] = 1
    label_prob = {}
    for i in label_count.keys():
        label_prob[i] = label_count[i] / len(data)
        
    A = np.zeros(1000, dtype=int)
    B = np.zeros(1000, dtype=int)
    E = np.zeros(1000, dtype=int)
    V = np.zeros(1000, dtype=int)
    for i in range(len(data)):
        if data[i][1] == 'B':
            B = np.sum([B,new_data[i]],axis=0)
        if data[i][1] == 'A':
            A = np.sum([A,new_data[i]],axis=0)
        if data[i][1] == 'E':
            E = np.sum([E,new_data[i]],axis=0)
        if data[i][1] == 'V':
            V = np.sum([V,new_data[i]],axis=0)
    A_count = sum(A)
    B_count = sum(B)
    E_count = sum(E)
    V_count = sum(V)
# calculate conditional probabilities
    conditional_probability_A = []
    conditional_probability_B = []
    conditional_probability_E = []
    conditional_probability_V = []
    for a in range(len(A)):
        probability_A = (A[a]+1)/(A_count+1000)
        conditional_probability_A.append(probability_A)
    for b in range(len(B)):
        probability_B = (B[b]+1)/(B_count+1000)
        conditional_probability_B.append(probability_B)
    for e in range(len(E)):
        probability_E = (E[e]+1)/(E_count+1000)
        conditional_probability_E.append(probability_E)
    for v in range(len(V)):
        probability_V = (V[v]+1)/(V_count+1000)
        conditional_probability_V.append(probability_V)
    return top1000_dictionary_id,label_prob, conditional_probability_A, conditional_probability_B, conditional_probability_E, conditional_probability_V



trained_model = NaiveBayes(data)
write_output(test_filename, output_filename, trained_model)

In [81]:
cv = 10
def test(abstract, trained_model):
    correct_predict = 0
    for i in range(len(abstract)):
        text = abstract[i][2].replace('"', ' ').split()
        prediction = predict(text, trained_model)
        if prediction == abstract[i][1]:
            correct_predict += 1
    acc =  correct_predict / len(abstract)
    return acc
        

def cross_validation(data,cv):
    acc_list = []
    each_fold_data_size = len(data) // cv
    start_index = list(range(0, len(data), each_fold_data_size))
    for i in range(cv):
        start = start_index[i]
        test_data= data[start: start + each_fold_data_size]
        train_data = np.append(data[:start], data[start + each_fold_data_size:], axis=0)
        model = NaiveBayes(train_data)
        acc = test(test_data, model)
        acc_list.append(acc)
    mean_score = sum(acc_list) / len(acc_list)
                       
                       
    return mean_score

acc_temp = cross_validation(data, cv)
acc_temp

0.905

In [82]:
data = np.genfromtxt(f'trg.csv', delimiter=',', encoding='utf8', dtype=None)[1:]

train_filename = "trg"
test_filename = "tst"
output_filename = "eliu092"

In [83]:
def write_output(test_filename, output_filename, trained_model):
    test_data = np.genfromtxt(f'{test_filename}.csv', delimiter=',', encoding='utf8', dtype=None)[1:]
    output = open(f'{output_filename}.csv', encoding='utf8', mode='w')
    output.write('id,class\n')
    for i in range(len(test_data)):
        abstract = test_data[i][1].replace('"', ' ').split()
        abstract = preprocessing(abstract)
        prediction = predict(abstract, trained_model)
        output.write(f'{i + 1},{prediction}\n')
    output.close()

def predict(abstract,trained_model):
        top1000_dictionary_id, label_prob, conditional_probabilities_A,conditional_probabilities_B,conditional_probabilities_E,conditional_probabilities_V = trained_model
        word_vector = np.zeros(1000, dtype=int)
        for word in abstract:
            if word in top1000_dictionary_id:
                word_vector[top1000_dictionary_id[word]] += 1
                
        
        p_label_A = np.log(label_prob['A'])
        p_A = sum(word_vector * np.log(conditional_probabilities_A)) + p_label_A
    
        p_label_B = np.log(label_prob['B'])
        p_B = sum(word_vector * np.log(conditional_probabilities_B)) + p_label_B
    
        p_label_E = np.log(label_prob['E'])
        p_E = sum(word_vector * np.log(conditional_probabilities_E)) + p_label_E
    
        p_label_V = np.log(label_prob['V'])
        p_V = sum(word_vector * np.log(conditional_probabilities_V)) + p_label_V
        prediction = {"A": p_A, "B": p_B, "E": p_E, "V": p_V}
        prediction_array = prediction.items()
        p = sorted(prediction_array, key=lambda key_value: key_value[1], reverse=True)
        p = p[0]
        return p[0]

def preprocessing(abstract):       
    stop_words = ['am','is','are','was','were','she','he','they','these','his','their','those','there','her','them','it','its','have','has',
                  'at','in','on','by','for','to','the','a','an','such','as','be','but','however','that','come','go','from','this','we','most',
                  'since','of','next','then','and','or','may','might','when','alse','some','with','which','who','whose','between','been','do','did','done'
                 ]
    abstract = [word for word in abstract if word not in stop_words]
    return abstract  
def Convert(tup, dictionary):
    i = 0
    for a, b in tup:
        dictionary.setdefault(a,i)
        i += 1
    return dictionary

def NaiveBayes(data):
    count_apperance = {}
    top1000 = []
    top1000_dictionary = {}
    for i in range(len(data)):
        abstract =  data[i][2]
        abstract = abstract.replace('"', ' ').split()
        abstract = preprocessing(abstract)
        for word in abstract:
            if word in count_apperance:
                count_apperance[word] += 1
            else:
                count_apperance[word] = 1
    count_apperance_array = count_apperance.items()    
    descending_sorting = sorted(count_apperance_array, key = lambda x:x[1], reverse = True)
    for i in range(0,1000):
        top1000.append(descending_sorting[i])
    top1000_dictionary_id =  Convert(top1000, top1000_dictionary)
    new_data = np.zeros((len(data), 1000))
    for i in range(len(data)):
        abstract = data[i][2].split()
        word_vector = np.zeros(1000, dtype=int)
        for word in abstract:
            if word in top1000_dictionary_id:
                word_vector[top1000_dictionary_id[word]] += 1
        new_data[i] = word_vector
# class probability calculation
    label_count = {}
    for i in range(len(data)):
        label = data[i][1]
        if label in label_count:
            label_count[label] += 1
        else:
            label_count[label] = 1
    label_prob = {}
    for i in label_count.keys():
        label_prob[i] = label_count[i] / len(data)
        
    A = np.zeros(1000, dtype=int)
    B = np.zeros(1000, dtype=int)
    E = np.zeros(1000, dtype=int)
    V = np.zeros(1000, dtype=int)
    for i in range(len(data)):
        if data[i][1] == 'B':
            B = np.sum([B,new_data[i]],axis=0)
        if data[i][1] == 'A':
            A = np.sum([A,new_data[i]],axis=0)
        if data[i][1] == 'E':
            E = np.sum([E,new_data[i]],axis=0)
        if data[i][1] == 'V':
            V = np.sum([V,new_data[i]],axis=0)
    A_count = sum(A)
    B_count = sum(B)
    E_count = sum(E)
    V_count = sum(V)
# calculate conditional probabilities
    conditional_probability_A = []
    conditional_probability_B = []
    conditional_probability_E = []
    conditional_probability_V = []
    for a in range(len(A)):
        probability_A = (A[a]+1)/(A_count+1000)
        conditional_probability_A.append(probability_A)
    for b in range(len(B)):
        probability_B = (B[b]+1)/(B_count+1000)
        conditional_probability_B.append(probability_B)
    for e in range(len(E)):
        probability_E = (E[e]+1)/(E_count+1000)
        conditional_probability_E.append(probability_E)
    for v in range(len(V)):
        probability_V = (V[v]+1)/(V_count+1000)
        conditional_probability_V.append(probability_V)
    return top1000_dictionary_id,label_prob, conditional_probability_A, conditional_probability_B, conditional_probability_E, conditional_probability_V



trained_model = NaiveBayes(data)
write_output(test_filename, output_filename, trained_model)

In [84]:
cv = 10
def test(abstract, trained_model):
    correct_predict = 0
    for i in range(len(abstract)):
        text = abstract[i][2].replace('"', ' ').split()
        word = preprocessing(text)
        prediction = predict(word, trained_model)
        if prediction == abstract[i][1]:
            correct_predict += 1
    acc =  correct_predict / len(abstract)
    return acc
        

def cross_validation(data,cv):
    acc_list = []
    each_fold_data_size = len(data) // cv
    start_index = list(range(0, len(data), each_fold_data_size))
    for i in range(cv):
        start = start_index[i]
        test_data= data[start: start + each_fold_data_size]
        train_data = np.append(data[:start], data[start + each_fold_data_size:], axis=0)
        model = NaiveBayes(train_data)
        acc = test(test_data, model)
        acc_list.append(acc)
    mean_score = sum(acc_list) / len(acc_list)
                       
                       
    return mean_score

acc_temp = cross_validation(data, cv)
acc_temp

0.907

In [37]:
data = np.genfromtxt(f'trg.csv', delimiter=',', encoding='utf8', dtype=None)[1:]

train_filename = "trg"
test_filename = "tst"
output_filename = "eliu092_update"

In [38]:
def write_output(test_filename, output_filename, trained_model):
    test_data = np.genfromtxt(f'{test_filename}.csv', delimiter=',', encoding='utf8', dtype=None)[1:]
    output = open(f'{output_filename}.csv', encoding='utf8', mode='w')
    output.write('id,class\n')
    for i in range(len(test_data)):
        abstract = test_data[i][1].replace('"', ' ').split()
        abstract = preprocessing(abstract)
        prediction = predict(abstract, trained_model)
        output.write(f'{i + 1},{prediction}\n')
    output.close()

def predict(abstract,trained_model):
        top1000_dictionary_id, label_prob, conditional_probabilities_A,conditional_probabilities_B,conditional_probabilities_E,conditional_probabilities_V = trained_model
        global count
        word_vector = np.zeros(count, dtype=int)
        for word in abstract:
            if word in top1000_dictionary_id:
                word_vector[top1000_dictionary_id[word]] += 1
                
        
        p_label_A = np.log(label_prob['A'])
        p_A = sum(word_vector * np.log(conditional_probabilities_A)) + p_label_A
    
        p_label_B = np.log(label_prob['B'])
        p_B = sum(word_vector * np.log(conditional_probabilities_B)) + p_label_B
    
        p_label_E = np.log(label_prob['E'])
        p_E = sum(word_vector * np.log(conditional_probabilities_E)) + p_label_E
    
        p_label_V = np.log(label_prob['V'])
        p_V = sum(word_vector * np.log(conditional_probabilities_V)) + p_label_V
        prediction = {"A": p_A, "B": p_B, "E": p_E, "V": p_V}
        prediction_array = prediction.items()
        p = sorted(prediction_array, key=lambda key_value: key_value[1], reverse=True)
        p = p[0]
        return p[0]

def preprocessing(abstract):       
    stop_words = ['am','is','are','was','were','she','he','they','these','his','their','those','there','her','them','it','its','have','has',
                  'at','in','on','by','for','to','the','a','an','such','as','be','but','however','that','come','go','from','this','we','most',
                  'since','of','next','then','and','or','may','might','when','alse','some','with','which','who','whose','1','2','3','also',
                  '4','5','6','7','8','9','0','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','between','been','do','did','done',
                  'p','q','r','s','t','u','v','w','x','y','z'
                 ]
    abstract = [word for word in abstract if word not in stop_words]
    return abstract  
def Convert(tup, dictionary):
    i = 0
    for a, b in tup:
        dictionary.setdefault(a,i)
        i += 1
    return dictionary

def NaiveBayes(data):
    count_apperance = {}
    top1000 = []
    top1000_dictionary = {}
    for i in range(len(data)):
        abstract =  data[i][2]
        abstract = abstract.replace('"', ' ').split()
        abstract = preprocessing(abstract)
        for word in abstract:
            if word in count_apperance:
                count_apperance[word] += 1
            else:
                count_apperance[word] = 1
    count_apperance_array = count_apperance.items()    
    descending_sorting = sorted(count_apperance_array, key = lambda x:x[1], reverse = True)
    global count
#     count = len(descending_sorting)
    count = 12000
    for i in range(0,count):
        top1000.append(descending_sorting[i])
    top1000_dictionary_id =  Convert(top1000, top1000_dictionary)
    new_data = np.zeros((len(data), count))
    for i in range(len(data)):
        abstract = data[i][2].split()
        word_vector = np.zeros(count, dtype=int)
        for word in abstract:
            if word in top1000_dictionary_id:
                word_vector[top1000_dictionary_id[word]] += 1
        new_data[i] = word_vector
# class probability calculation
    label_count = {}
    for i in range(len(data)):
        label = data[i][1]
        if label in label_count:
            label_count[label] += 1
        else:
            label_count[label] = 1
    label_prob = {}
    for i in label_count.keys():
        label_prob[i] = label_count[i] / len(data)
        
    A = np.zeros(count, dtype=int)
    B = np.zeros(count, dtype=int)
    E = np.zeros(count, dtype=int)
    V = np.zeros(count, dtype=int)
    for i in range(len(data)):
        if data[i][1] == 'B':
            B = np.sum([B,new_data[i]],axis=0)
        if data[i][1] == 'A':
            A = np.sum([A,new_data[i]],axis=0)
        if data[i][1] == 'E':
            E = np.sum([E,new_data[i]],axis=0)
        if data[i][1] == 'V':
            V = np.sum([V,new_data[i]],axis=0)
    A_count = sum(A)
    B_count = sum(B)
    E_count = sum(E)
    V_count = sum(V)
# calculate conditional probabilities
    conditional_probability_A = []
    conditional_probability_B = []
    conditional_probability_E = []
    conditional_probability_V = []
    for a in range(len(A)):
        probability_A = (A[a]+1)/(A_count+count)
        conditional_probability_A.append(probability_A)
    for b in range(len(B)):
        probability_B = (B[b]+1)/(B_count+count)
        conditional_probability_B.append(probability_B)
    for e in range(len(E)):
        probability_E = (E[e]+1)/(E_count+count)
        conditional_probability_E.append(probability_E)
    for v in range(len(V)):
        probability_V = (V[v]+1)/(V_count+count)
        conditional_probability_V.append(probability_V)
    return top1000_dictionary_id,label_prob, conditional_probability_A, conditional_probability_B, conditional_probability_E, conditional_probability_V



trained_model = NaiveBayes(data)
write_output(test_filename, output_filename, trained_model)

In [39]:
cv = 10
def test(abstract, trained_model):
    correct_predict = 0
    for i in range(len(abstract)):
        text = abstract[i][2].replace('"', ' ').split()
        word = preprocessing(text)
        prediction = predict(word, trained_model)
        if prediction == abstract[i][1]:
            correct_predict += 1
    acc =  correct_predict / len(abstract)
    return acc
        

def cross_validation(data,cv):
    acc_list = []
    each_fold_data_size = len(data) // cv
    start_index = list(range(0, len(data), each_fold_data_size))
    for i in range(cv):
        start = start_index[i]
        test_data= data[start: start + each_fold_data_size]
        train_data = np.append(data[:start], data[start + each_fold_data_size:], axis=0)
        model = NaiveBayes(train_data)
        acc = test(test_data, model)
        acc_list.append(acc)
    mean_score = sum(acc_list) / len(acc_list)
                       
                       
    return mean_score

acc_temp = cross_validation(data, cv)
acc_temp

0.95525

In [49]:
data = np.genfromtxt(f'trg.csv', delimiter=',', encoding='utf8', dtype=None)[1:]
train_filename = "trg"
test_filename = "tst"
output_filename = "eliu092_final_ff"

In [51]:
def write_output(test_filename, output_filename, trained_model):
    test_data = np.genfromtxt(f'{test_filename}.csv', delimiter=',', encoding='utf8', dtype=None)[1:]
    output = open(f'{output_filename}.csv', encoding='utf8', mode='w')
    output.write('id,class\n')
    for i in range(len(test_data)):
        abstract = test_data[i][1].replace('"', ' ').split()
        abstract = preprocessing(abstract)
        prediction = predict(abstract, trained_model)
        output.write(f'{i + 1},{prediction}\n')
    output.close()

def predict(abstract,trained_model):
        top1000_dictionary_id, label_prob, conditional_probabilities_A,conditional_probabilities_B,conditional_probabilities_E,conditional_probabilities_V = trained_model
        global count
        word_vector = np.zeros(count, dtype=int)
        for word in abstract:
            if word in top1000_dictionary_id:
                word_vector[top1000_dictionary_id[word]] += 1
                
        
        p_label_A = np.log(label_prob['A'])
        p_A = sum(word_vector * np.log(conditional_probabilities_A)) + p_label_A
    
        p_label_B = np.log(label_prob['B'])
        p_B = sum(word_vector * np.log(conditional_probabilities_B)) + p_label_B
    
        p_label_E = np.log(label_prob['E'])
        p_E = sum(word_vector * np.log(conditional_probabilities_E)) + p_label_E
    
        p_label_V = np.log(label_prob['V'])
        p_V = sum(word_vector * np.log(conditional_probabilities_V)) + p_label_V
        prediction = {"A": p_A, "B": p_B, "E": p_E, "V": p_V}
        prediction_array = prediction.items()
        p = sorted(prediction_array, key=lambda key_value: key_value[1], reverse=True)
        p = p[0]
        return p[0]

def preprocessing(abstract):       
    stop_words = ['am','is','are','was','were','she','he','they','these','his','their','those','there','her','them','it','its','have','has',
                  'at','in','on','by','for','to','the','a','an','such','as','be','but','however','that','come','go','from','this','we','most',
                  'since','of','next','then','and','or','may','might','when','alse','some','with','which','who','whose','also','between','been','do','did','done'
                 ]
    abstract = [word for word in abstract if word not in stop_words]
    return abstract  
def Convert(tup, dictionary):
    i = 0
    for a, b in tup:
        dictionary.setdefault(a,i)
        i += 1
    return dictionary

def NaiveBayes(data):
    count_apperance = {}
    top1000 = []
    top1000_dictionary = {}
    for i in range(len(data)):
        abstract =  data[i][2]
        abstract = abstract.replace('"', ' ').split()
        abstract = preprocessing(abstract)
        for word in abstract:
            if word in count_apperance:
                count_apperance[word] += 1
            else:
                count_apperance[word] = 1
    count_apperance_array = count_apperance.items()    
    descending_sorting = sorted(count_apperance_array, key = lambda x:x[1], reverse = True)
    global count
#     count = len(descending_sorting)
    count = 12000
    for i in range(0,count):
        top1000.append(descending_sorting[i])
    top1000_dictionary_id =  Convert(top1000, top1000_dictionary)
    new_data = np.zeros((len(data), count))
    for i in range(len(data)):
        abstract = data[i][2].split()
        word_vector = np.zeros(count, dtype=int)
        for word in abstract:
            if word in top1000_dictionary_id:
                word_vector[top1000_dictionary_id[word]] += 1
        new_data[i] = word_vector
#TF IDF
    len_data_instance = len(new_data)
    new_data = np.transpose(new_data)
    for i in range(count):
        count_frequency = len([1 for j in new_data[i] if j != 0])
        new_data[i] = new_data[i] * np.log(len_data_instance / (count_frequency + 1) )
    new_data = np.transpose(new_data)
    
    
# class probability calculation
    label_count = {}
    for i in range(len(data)):
        label = data[i][1]
        if label in label_count:
            label_count[label] += 1
        else:
            label_count[label] = 1
    label_prob = {}
    for i in label_count.keys():
        label_prob[i] = label_count[i] / len(data)
        
    A = np.zeros(count, dtype=int)
    B = np.zeros(count, dtype=int)
    E = np.zeros(count, dtype=int)
    V = np.zeros(count, dtype=int)
    for i in range(len(data)):
        if data[i][1] == 'B':
            B = np.sum([B,new_data[i]],axis=0)
        if data[i][1] == 'A':
            A = np.sum([A,new_data[i]],axis=0)
        if data[i][1] == 'E':
            E = np.sum([E,new_data[i]],axis=0)
        if data[i][1] == 'V':
            V = np.sum([V,new_data[i]],axis=0)
    A_count = sum(A)
    B_count = sum(B)
    E_count = sum(E)
    V_count = sum(V)
# calculate conditional probabilities
    conditional_probability_A = []
    conditional_probability_B = []
    conditional_probability_E = []
    conditional_probability_V = []
    for a in range(len(A)):
        probability_A = (A[a]+1)/(A_count+count)
        conditional_probability_A.append(probability_A)
    for b in range(len(B)):
        probability_B = (B[b]+1)/(B_count+count)
        conditional_probability_B.append(probability_B)
    for e in range(len(E)):
        probability_E = (E[e]+1)/(E_count+count)
        conditional_probability_E.append(probability_E)
    for v in range(len(V)):
        probability_V = (V[v]+1)/(V_count+count)
        conditional_probability_V.append(probability_V)
    return top1000_dictionary_id,label_prob, conditional_probability_A, conditional_probability_B, conditional_probability_E, conditional_probability_V



trained_model = NaiveBayes(data)
write_output(test_filename, output_filename, trained_model)

In [52]:
cv = 10
def test(abstract, trained_model):
    correct_predict = 0
    for i in range(len(abstract)):
        text = abstract[i][2].replace('"', ' ').split()
        word = preprocessing(text)
        prediction = predict(word, trained_model)
        if prediction == abstract[i][1]:
            correct_predict += 1
    acc =  correct_predict / len(abstract)
    return acc
        

def cross_validation(data,cv):
    acc_list = []
    each_fold_data_size = len(data) // cv
    start_index = list(range(0, len(data), each_fold_data_size))
    for i in range(cv):
        start = start_index[i]
        test_data= data[start: start + each_fold_data_size]
        train_data = np.append(data[:start], data[start + each_fold_data_size:], axis=0)
        model = NaiveBayes(train_data)
        acc = test(test_data, model)
        acc_list.append(acc)
    mean_score = sum(acc_list) / len(acc_list)
                       
                       
    return mean_score

acc_temp = cross_validation(data, cv)
acc_temp

0.96225